# 第五章 例5-2-1 Least square balance method 最小二乘法平衡SAM表

In [56]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Domain,Sense
import pandas as pd
pd.options.display.float_format = '{:.0f}'.format

### Container

In [57]:
m = Container()

### Sets

In [58]:
section = ["sec1","sec2","lab","hh"]
i = Set(container=m, name="i", description="单位", records=section)
j = Alias(m, name="j", alias_with=i)
i.name, j.name, i.records

('i',
 'j',
     uni element_text
 0  sec1             
 1  sec2             
 2   lab             
 3    hh             )

### Data

In [59]:
data = pd.read_excel("ch5-2-1.xlsx", index_col=0)
data = data.fillna(0)
data

,sec1,sec2,lab,hh,total
sec1,52,45,0,150,247
sec2,95,48,0,90,233
lab,120,89,0,0,209
hh,0,0,192,0,192
total,267,182,192,240,0


In [60]:
Q0_data = data.loc[section,section].stack().reset_index()


### Parameters

In [61]:
Q0 = Parameter(
    container=m,
    name="Q0",
    domain=[i,j],
    description="intial value",
    records=Q0_data
)
Q0.records


,level_0,level_1,value
0,sec1,sec1,52
1,sec1,sec2,45
2,sec1,lab,0
3,sec1,hh,150
4,sec2,sec1,95
5,sec2,sec2,48
6,sec2,lab,0
7,sec2,hh,90
8,lab,sec1,120
9,lab,sec2,89


### Variables

In [62]:
Q = Variable(
    container=m,
    name="Q",
    domain=[i,j],
    type="Positive",
    description="adjusted value",
)

### Equations

In [63]:
balance = Equation( 
    container=m, name="balance", domain=i,description="balance equation"  
)
balance[i] = Sum(j.where[Q0[i,j]>0], Q[i,j])  == Sum(j.where[Q0[j,i]>0], Q[j,i])
print(balance.latexRepr())

$
\sum_{j ~ | ~ Q0_{i,j} > 0} Q_{i,j} = \sum_{j ~ | ~ Q0_{j,i} > 0} Q_{j,i}\hfill \forall i
$


### Objective

In [64]:
obj = Sum(Domain(i, j).where[Q0[i,j]>0],(Q[i,j]-Q0[i,j])**2 )

### Model

In [65]:
sambal = Model(
    m,
    name="sambal",
    equations=m.getEquations(),
    problem="NLP",
    sense=Sense.MIN,
    objective=obj,
)
Q.l[i,j] = Q0[i,j]

### Solve


In [66]:
sambal.solve()
Q.records


,i,j,level,marginal,lower,upper,scale
0,sec1,sec1,52,0,0,inf,1
1,sec1,sec2,57,0,0,inf,1
2,sec1,hh,140,0,0,inf,1
3,sec2,sec1,83,0,0,inf,1
4,sec2,sec2,48,0,0,inf,1
5,sec2,hh,68,0,0,inf,1
6,lab,sec1,114,0,0,inf,1
7,lab,sec2,95,0,0,inf,1
8,hh,lab,208,0,0,inf,1


In [67]:
sambal.objective_value

1188.4583333333337

In [68]:
results = Q.records.loc[:,["i","j","level"]].pivot(index="i",columns="j",values="level")
results

j,sec1,sec2,lab,hh
i,,,,
sec1,52,57,NaN,140
sec2,83,48,NaN,68
lab,114,95,NaN,NaN
hh,NaN,NaN,208,NaN
